# Gestionando datos perdidos

Ocurre con frecuencia, y desgraciadamente, la obtención de datos anómalos o incluso la ausencia de muestras.

Por ejemplo, los datos obtenidos a partir de encuestas: preguntas sin responder, preguntas con respuestas "raras", etc.

**Hay que aceptarlo y saber gestionarlo**

En Pandas, los valores perdidos se les asigna el código: NaN (Not a Number), los objetos son designados como: None y las fechas como NaT.

Las operaciones que gestionen este tipo de datos también generarán los correspondientes códigos: NaN, None o NaT.

¡Vamos a trabajar con estos valores!



In [4]:
import pandas as pd

## Contenido

- M2_0 Introducción a la estructura DataFrame: características, carga y acceso.**
- M2_1 Creación y Almacenamiento.
- M2_2 Visualización con pandas.
- M2_3 Otras operaciones con DataFrames: agrupaciones de datos.
- **M2_4 Gestionando datos perdidos**.
- M2_5 Series temporales.

In [5]:
#Empezamos cargando datos: who.csv con 358 columnas!
df = pd.read_csv("data/who.csv")
df = df[["Country",df.columns[-2]]]
print(df[:5])

       Country  Urban_population_growth
0  Afghanistan                     5.44
1      Albania                     2.21
2      Algeria                     2.61
3      Andorra                      NaN
4       Angola                     4.14


In [6]:
# Atención: Si el fichero es MUY pesado es recomendable cargar solo la información que nos interesa
# https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_csv.html
df = pd.read_csv("data/who.csv", usecols=["Country","Urban_population_growth"])
print(df[:5])

       Country  Urban_population_growth
0  Afghanistan                     5.44
1      Albania                     2.21
2      Algeria                     2.61
3      Andorra                      NaN
4       Angola                     4.14


In [8]:
#¿Qué columnas tienen datos "NaX"?
# https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.isna.html
print(df.columns[df.isna().any()])

Index(['Urban_population_growth'], dtype='object')


In [31]:
#No dudéis en ejecutar "partes" (dividamos la instrucción para comprenderla)
print df.isna()[:5]

   Country  Urban_population_growth
0    False                    False
1    False                    False
2    False                    False
3    False                     True
4    False                    False


In [36]:
#¿Cuántas muestras son correctas? 
print pd.notna(df).sum() #**
print "Total de muestras: ",len(df)

Country                    202
Urban_population_growth    188
dtype: int64
Total de muestras:  202


In [39]:
#La manera más optima de remplazar estos valores es con la función: fillna
print df.fillna(0)[:5]


       Country  Urban_population_growth
0  Afghanistan                     5.44
1      Albania                     2.21
2      Algeria                     2.61
3      Andorra                     0.00
4       Angola                     4.14


In [40]:
df = df.fillna(0) #Atención: recordad que tenéis que hacer la asignación si queréis aplicar la operación

#### Cuando los dataframes contienen números la operabildad con valores perdidos puede gestionarse de manera más eficiente
Pongamos un ejemplo

In [26]:
import numpy as np
df.two = pd.DataFrame(np.random.randn(5, 3), 
                     index=['a', 'b', 'c', 'd', 'e'],
                     columns=['one', 'two', 'three'])
print(df.two)

        one       two     three
a  0.958487  0.961709 -0.423418
b -3.341468  1.917047 -0.374298
c -1.176809  1.414488 -0.725104
d  0.185405  0.256344  0.446815
e  0.189223  0.372475 -0.191705


In [27]:
#Creamos valores NaN para testear 
df.two[df.two<0]=np.nan
print(df.two)

        one       two     three
a  0.958487  0.961709       NaN
b       NaN  1.917047       NaN
c       NaN  1.414488       NaN
d  0.185405  0.256344  0.446815
e  0.189223  0.372475       NaN


In [30]:
# y por si las moscas, no nos fiémos del aleatorio.
df.iloc[0,0] = np.nan
print(df)

                Country  Urban_population_growth
0                   NaN                     5.44
1               Albania                     2.21
2               Algeria                     2.61
3               Andorra                      NaN
4                Angola                     4.14
..                  ...                      ...
197             Vietnam                     2.90
198  West Bank and Gaza                     3.33
199               Yemen                     4.37
200              Zambia                     1.95
201            Zimbabwe                     1.90

[202 rows x 2 columns]


#### Podemos usar fillNA para rellenar de manera elegante los valores. Por ejemplo, usando la media

In [32]:
print(df)
print("-"*33)
print(df.fillna(df.mean()))

                Country  Urban_population_growth
0                   NaN                     5.44
1               Albania                     2.21
2               Algeria                     2.61
3               Andorra                      NaN
4                Angola                     4.14
..                  ...                      ...
197             Vietnam                     2.90
198  West Bank and Gaza                     3.33
199               Yemen                     4.37
200              Zambia                     1.95
201            Zimbabwe                     1.90

[202 rows x 2 columns]
---------------------------------
                Country  Urban_population_growth
0                   NaN                 5.440000
1               Albania                 2.210000
2               Algeria                 2.610000
3               Andorra                 2.165851
4                Angola                 4.140000
..                  ...                      ...
197        

In [34]:
#Con la media de un valor en concreto
df.fillna(df.loc[0, ["Urban_population_growth"]])

,Country,Urban_population_growth
0,NaN,5.44
1,Albania,2.21
2,Algeria,2.61
3,Andorra,5.44
4,Angola,4.14
...,...,...
197,Vietnam,2.90
198,West Bank and Gaza,3.33
199,Yemen,4.37
200,Zambia,1.95


**Break conceptual:**

Hasta ahora solo habíamos visto dataframes con su índice numérico. El dataframe *df.two* es diferente

In [37]:
df.two.index

Index(['a', 'b', 'c', 'd', 'e'], dtype='object')

In [36]:
df.two.loc["a", df.two.columns.values]

one      0.958487
two      0.961709
three    3.000000
Name: a, dtype: float64

## Eliminación de datos perdidos: Otra manera de operar con ellos

En primer lugar eliminaremos las filas que contienen valores faltantes, luego operaremos con las columnas.

In [35]:
df.two["three"]=[3]*5
#[3]*5 == [3,3,3,3,3]
print(df.two)
print("-"*35)
print(df.two.dropna())

        one       two  three
a  0.958487  0.961709      3
b       NaN  1.917047      3
c       NaN  1.414488      3
d  0.185405  0.256344      3
e  0.189223  0.372475      3
-----------------------------------
        one       two  three
a  0.958487  0.961709      3
d  0.185405  0.256344      3
e  0.189223  0.372475      3


In [38]:
#Otra manera de borrar los datos es considerando las coumnas (axis=1) por defecto es axis=0
df.two.dropna(axis=1)

,two,three
a,0.961709,3
b,1.917047,3
c,1.414488,3
d,0.256344,3
e,0.372475,3


**La interpolación sirve como metodo para tratar con valores desconocidos.**

In [39]:
print(df.two)
print("-"*35)
print(df.two.interpolate())

        one       two  three
a  0.958487  0.961709      3
b       NaN  1.917047      3
c       NaN  1.414488      3
d  0.185405  0.256344      3
e  0.189223  0.372475      3
-----------------------------------
        one       two  three
a  0.958487  0.961709      3
b  0.700793  1.917047      3
c  0.443099  1.414488      3
d  0.185405  0.256344      3
e  0.189223  0.372475      3


In [40]:
print("Valores interpolados:" + str(df.two.interpolate().count()-df.two.count()))

Valores interpolados:one      2
two      0
three    0
dtype: int64


### Hay múltiples maneras de interpolar:

En la documentación vemos una serie de ejemplos: <a target="_blank" href="https://pandas.pydata.org/pandas-docs/stable/generated/pandas.Series.interpolate.html"> Interpolate </a> 

# Ejercicios de gestión de valores perdidos

**1) Del fichero who.csv, contabiliza cuántos paises tienen algun valor NaN.**

**1b) Ordena el anterior resultado para identificar cuál es el pais con mayor número de campos desconocidos.**

**2) who.csv, Selecciona la primera, tercera y decima columna, de las filas comprendidas entre la 100 y la 150.**

**2b) ¿Cuántos valores NaN hay presentes?**

**2c) Crea un nuevo dataframe donde los NaN sean cero.**

**2d) Elimina aquellas filas de la anterior selección donde haya NaN.**

**3) Genera un dataframe de 5 x 100, con números aleatorios entre 0-10. Las celdas que superen un valor de >=8 se conviertan en valores NaN.**

**3a) Interpola los anteriores valores mediante interpolación simple.**

**3b) Interpola la serie 3, hasta un limite de 10 interpolaciones, con el metodo del más cercano "nearest".**